# Opnepyxlを利用したエクセルデータの読み書き

公式  
https://openpyxl.readthedocs.io/en/stable/index.html

### ライブラリのインストール
pip install openpyxl  
でインストールが必要

In [1]:
# 必要ライブラリのインポート
from openpyxl import Workbook, load_workbook

# データの加工編集用にpandasを利用
import pandas as pd

### データの読み込み

In [2]:
# エクセルファイルを読込む
data_wb = load_workbook("../input/データ分割後のデータサンプル.xlsx",data_only=True)#data_onlyがTrueでないと数式セルは数値でなく数式が読み込まれる

In [3]:
# 特定のシートの特定の表の範囲を取得
sheet_name = '処方'
table_name = '処方'
table_range = data_wb[sheet_name].tables[table_name].ref
table_range


'B6:R39'

In [4]:
# 読込んだテーブルの範囲からデータをpandasの形式で読み込み
data_list = []
for row in data_wb[sheet_name][table_range]:#データを市行ごとに取りだし
    data_list.append([cell.value for cell in row])#セルオブジェクトから1セルごとに値を取得
syoho_df = pd.DataFrame(data_list[1:],columns=data_list[0])
syoho_df.head()

,塗料サンプルID,サンプル作成日,担当者,備考,材料１,材料２,材料３,材料４,材料５,材料６,材料７,材料８,材料９,材料１０,材料１１,材料１２,材料１３
0,DP_001,2024-01-19,高岡,None,46.9,40,0,2,0,2.0,0.0,0.0,5.0,0.0,4,0,0.1
1,DP_002,2024-01-19,高岡,None,47.4,40,0,2,0,1.5,0.0,0.0,5.0,0.0,4,0,0.1
2,DP_003,2024-01-19,高岡,None,47.7,40,0,2,0,1.2,0.0,0.0,5.0,0.0,4,0,0.1
3,DP_004,2024-01-19,高岡,None,47.9,40,0,2,0,1.0,0.0,0.0,5.0,0.0,4,0,0.1
4,DP_005,2024-01-19,高岡,None,48.4,40,0,2,0,0.5,0.0,0.0,5.0,0.0,4,0,0.1


In [5]:
# 特定のシートの特定の表を取得
sheet_name2 = '表面張力'
table_name2 = '表面張力'
table_range2 = data_wb[sheet_name2].tables[table_name2].ref

data_list = []
for row in data_wb[sheet_name2][table_range2]:
    data_list.append([cell.value for cell in row])
st_df = pd.DataFrame(data_list[1:],columns=data_list[0])
st_df.head()

,表面張力評価ID,塗料サンプルID,備考,評価日,測定機器No,測定温度,表面張力
0,ST_001,DP_001,None,2024-01-19,HT_01,25,35
1,ST_002,DP_002,None,2024-01-19,HT_01,25,38
2,ST_003,DP_003,None,2024-01-19,HT_01,25,40
3,ST_004,DP_004,None,2024-01-19,HT_01,25,41
4,ST_005,DP_005,None,2024-01-19,HT_01,25,44


### データの書き込み

In [6]:
all_df = pd.merge(syoho_df,st_df[['塗料サンプルID','表面張力']],on='塗料サンプルID',how='left')#二つの表を結合して一つに
all_df.head()

,塗料サンプルID,サンプル作成日,担当者,備考,材料１,材料２,材料３,材料４,材料５,材料６,材料７,材料８,材料９,材料１０,材料１１,材料１２,材料１３,表面張力
0,DP_001,2024-01-19,高岡,None,46.9,40,0,2,0,2.0,0.0,0.0,5.0,0.0,4,0,0.1,35
1,DP_002,2024-01-19,高岡,None,47.4,40,0,2,0,1.5,0.0,0.0,5.0,0.0,4,0,0.1,38
2,DP_003,2024-01-19,高岡,None,47.7,40,0,2,0,1.2,0.0,0.0,5.0,0.0,4,0,0.1,40
3,DP_004,2024-01-19,高岡,None,47.9,40,0,2,0,1.0,0.0,0.0,5.0,0.0,4,0,0.1,41
4,DP_005,2024-01-19,高岡,None,48.4,40,0,2,0,0.5,0.0,0.0,5.0,0.0,4,0,0.1,44


In [7]:
# データフレームを一度リストに変換
all_df_list = all_df.T.reset_index().T.values.tolist()

In [8]:
# 保存用の器を用意
wb = Workbook()
ws = wb.active #名称「Sheet」のシートが選択される

In [9]:
# データの書き込み
i_0 = 5#データ記入開始行
j_0 = 2#データ記入開始列
for i,row in enumerate(all_df_list):
    for j,v in enumerate(row):
        ws.cell(row=i+i_0, column=j+j_0, value=v)

In [11]:
wb.save("../output/openpyxlを用いたデータ出力のサンプル.xlsx")